<a href="https://colab.research.google.com/github/anshikabajpai23/Covid19_prediction/blob/main/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# @markdown Loading Dependencies
import os
from google.colab import files
!pip install -q kaggle
import shutil
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from zipfile import ZipFile
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from keras.models import Sequential, load_model
from keras.layers import (GRU, LSTM, Bidirectional, Dense, Dropout, Flatten,
                          SimpleRNN)
from keras.optimizers import (SGD, RMSprop, Adagrad,
                              Adadelta, Adam, Adamax, Nadam)
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import mean_squared_error,mean_absolute_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials,space_eval
from keras.models import load_model,save_model

try:
  !mkdir models
except:
  pass

Using TensorFlow backend.


In [ ]:
# @markdown importing dataset from kaggle
src = os.path.join('/content/gdrive/My Drive/covid-19', 'kaggle.json')
dst = os.path.join('/content', 'kaggle.json')
shutil.copy(src, dst)
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download -d sudalairajkumar/covid19-in-india


with ZipFile('/content/covid19-in-india.zip', 'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

data = pd.read_csv("/content/covid_19_india.csv")

  0% 0.00/251k [00:00<?, ?B/s]
100% 251k/251k [00:00<00:00, 37.2MB/s]
Extracting all the files now...
Done!


In [ ]:
data.tail(3)

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
4458,4459,24/07/20,8:00 AM,Uttarakhand,-,-,3399,60,5445
4459,4460,24/07/20,8:00 AM,Uttar Pradesh,-,-,35803,1289,58104
4460,4461,24/07/20,8:00 AM,West Bengal,-,-,31656,1255,51757


In [ ]:
# @markdown Dataset Preparation
Dates = []
for i in data['Date']:
  if i not in Dates:
    Dates.append(i)

Date=[]
Confirmed_cases = []
for i in Dates:
  df = data[data['Date']==i]
  Date.append(i)
  Confirmed_cases.append(df['Confirmed'].sum())

data_new = {'Date' : Date, 'Confirmed Cases': Confirmed_cases}
df_train = pd.DataFrame(data_new)

# Print the output.
#df_train=df_train.set_index('Date')

df_train.tail(2)

,Date,Confirmed Cases
175,23/07/20,1238635
176,24/07/20,1287945


In [ ]:
df_train =df_train.head(len(df_train.index)-1)
df_train.tail()

,Date,Confirmed Cases
171,19/07/20,1077618
172,20/07/20,1118043
173,21/07/20,1155191
174,22/07/20,1192915
175,23/07/20,1238635


In [ ]:
# @markdown Visualization 1

fig = go.Figure()
trace = fig.add_trace(go.Scatter(x=df_train['Date'], y=df_train['Confirmed Cases'], name='Confirmed Cases in India', mode='lines+markers',
                         line_color='#DC143C',line_width=1.5, hovertext=df_train['Confirmed Cases']))
fig.show()

In [ ]:
# @markdown Visualization 2
fig2 = px.bar(df_train, x="Date", y="Confirmed Cases", color='Confirmed Cases', orientation='v', height=600,
             title='Confirmed Cases in India', color_discrete_sequence = px.colors.cyclical.Twilight)
fig2.show()

In [ ]:
# @markdown Data Preprocessing
val = df_train['Confirmed Cases'].values
val_index = df_train['Date'].values
val = val.astype('float32')
#print(val_index)
#print(val)
val = np.reshape(val, (-1, 1))
#print(val)
scaler = MinMaxScaler(feature_range=(0, 1))
val = scaler.fit_transform(val)
#print(val)

In [ ]:
# @markdown Preparation of training and testing data
train_size = int(len(val) * 0.75)
print(train_size)
test_size = len(val) - train_size
print(test_size)
train, test = val[0:train_size,:], val[train_size:len(val),:]
index_train, index_test = val_index[0:train_size], val_index[train_size:len(val_index)]

def create_dataset(dataset, look_back, index_dataset):
    X, Y ,index_X,index_Y= [], [], [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        #print('x is : {}'.format(X))
        Y.append(dataset[i + look_back, 0])
        index_Y.append(index_dataset[i + look_back])
        #print('y is : {}'.format(Y))
        c = index_dataset[i:(i+look_back)]
        index_X.append(c)
    return np.array(X), np.array(Y),np.array(index_X),np.array(index_Y)

look_back = 10

X_train, Y_train, index_X_train, index_Y_train = create_dataset(train, look_back, index_train)
X_test, Y_test, index_X_test, index_Y_test = create_dataset(test, look_back, index_test)

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

132
44


In [ ]:
len(X_train)

121

In [ ]:
len(X_test)

33

In [ ]:
model= Sequential()
input_shape=(X_train.shape[1], X_train.shape[2])

In [ ]:
# @markdown RNN Model Training

model.add(SimpleRNN(
                200,
                return_sequences=True,
                activation='relu',
                input_shape=input_shape
                ))

model.add(Dropout(0.1))
model.add(SimpleRNN(
                    300,
                    return_sequences=True,
                    activation='tanh'
                    )
                )
model.add(Dropout(0.1))
model.add(SimpleRNN(
                100,
                return_sequences=True,
                activation='relu'
                )
            )
model.add(Flatten())


model.add(Dense(1, activation='linear',))

model.compile(loss='mean_squared_error', optimizer='Adam')
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]
history = model.fit(X_train, Y_train,
                    epochs=80,
                    batch_size=50,
                    callbacks=callbacks,
                    verbose=0,
                    shuffle=False)



In [ ]:
model = load_model('/content/models/modelwithloss557.h5')


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
# @markdown Performing predictions on training and testing
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)
train_predict = scaler.inverse_transform(train_predict)
Y_train_ = scaler.inverse_transform([Y_train])
test_predict = scaler.inverse_transform(test_predict)
Y_test_ = scaler.inverse_transform([Y_test])
print('Train Mean Absolute Error:', mean_absolute_error(Y_train_[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train_[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test_[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test_[0], test_predict[:,0])))

Train Mean Absolute Error: 1729.022779306459
Train Root Mean Squared Error: 2073.0834811262484
Test Mean Absolute Error: 2970.536090088592
Test Root Mean Squared Error: 4741.507467306251


In [ ]:
accuracy = 100 -(mean_absolute_percentage_error(Y_test_[0], test_predict[:,0]))
accuracy

99.6340494254643

In [ ]:
# @markdown Post Processing
train_pred = list(train_predict.flatten())
y_train = list(Y_train_.flatten())
y_test = list(Y_test_.flatten())
test_pred = list(test_predict.flatten())

for i in range(len(y_train)):
  train_pred[i] = int(round(train_pred[i]))
  y_train[i] = int(round(y_train[i]))
for i in range(len(y_test)):
  test_pred[i] = int(round(test_pred[i]))
  y_test[i] = int(round(y_test[i]))

dict_train = {'Date': list(index_Y_train), 'Actual': y_train,'Predicted' :train_pred}
dict_test = {'Date': list(index_Y_test), 'Actual': y_test ,'Predicted' :test_pred}

train_df = pd.DataFrame(dict_train)
test_df = pd.DataFrame(dict_test)

test_df.tail(5)

,Date,Actual,Predicted
28,18/07/20,1038716,1033930
29,19/07/20,1077618,1068253
30,20/07/20,1118043,1104104
31,21/07/20,1155191,1141700
32,22/07/20,1192915,1180074


In [ ]:
# @markdown For Training
fig = go.Figure()
trace1 = fig.add_trace(go.Scatter(x=train_df['Date'], y=train_df['Actual'], name='Actual', mode='lines+markers',
                         line_color='#DC143C',line_width=1.5, hovertext=train_df['Actual']))
trace2 = fig.add_trace(go.Scatter(x=train_df['Date'], y=train_df['Predicted'], name='Predicted', mode='lines+markers',
                         line_color='#49a7c7',line_width=1.5, hovertext=train_df['Predicted']))
fig.show()

In [ ]:
# @markdown For Testing

fig2 = go.Figure()
trace1 = fig2.add_trace(go.Scatter(x=test_df['Date'], y=test_df['Actual'], name='Actual', mode='lines+markers',
                         line_color='#DC143C',line_width=1.5, hovertext=test_df['Actual']))
trace2 = fig2.add_trace(go.Scatter(x=test_df['Date'], y=test_df['Predicted'], name='Predicted', mode='lines+markers',
                         line_color='#49a7c7',line_width=1.5, hovertext=test_df['Predicted']))
fig2.show()

# Performing Hyperparameter Tuning
Bayesian optimization using HyperOpt library(Tpe algorithm)

In [ ]:
# @markdown Objective Function
def createModel(space):
  model= Sequential()
  input_shape=(X_train.shape[1], X_train.shape[2])
  print(input_shape)
  model.add(SimpleRNN(space['neuron0'],
                      return_sequences=True,
                      activation = space['activation0'],
                      input_shape=input_shape))
  model.add(Dropout(space['dropout0']))
  model.add(SimpleRNN(space['neuron1'],return_sequences=True,activation = space['activation1'],input_shape=input_shape))
  model.add(Dropout(space['dropout1']))
  model.add(SimpleRNN(space['neuron2'],return_sequences=True,activation = space['activation2'],input_shape=input_shape))
  model.add(Flatten())
  model.add(Dense(1, activation='linear',))
  model.compile(loss='mean_squared_error', optimizer='Adam')
  callbacks = [ModelCheckpoint(monitor='val_loss', filepath='/content/mymodel.h5', save_best_only=False,save_weights_only=False)]
  history = model.fit(X_train, Y_train,epochs=space['epochs'],batch_size=space['batch_size'],callbacks=callbacks,verbose=1,shuffle=False)
  test_predict = model.predict(X_test)
  test_predict = scaler.inverse_transform(test_predict)
  Y_test_ = scaler.inverse_transform([Y_test])
  mae = mean_absolute_error(Y_test_[0], test_predict[:,0])
  mse = np.sqrt(mean_squared_error(Y_test_[0], test_predict[:,0]))
  round_mae = int(round(mae))
  file_path = '/content/models/modelwithloss' + str(round_mae) + '.h5'
  model.save(file_path)
  return {'loss': mae, 'status': STATUS_OK}


In [ ]:
# @markdown Definition of Space
space = {
    'neuron0' : hp.choice('neurons0',[50,100,150,200,250,300]),
    'neuron1' : hp.choice('neurons1',[50,100,150,200,250,300]),
    'neuron2' : hp.choice('neurons2',[50,100,150,200,250,300]),
    'dropout0' : hp.choice('dropout0',[0.1,0.2,0.3,0.4,0.5]),
    'dropout1' : hp.choice('dropout1',[0.1,0.2,0.3,0.4,0.5]),
    'epochs' : hp.choice('epochs',[20,50,80,100]),
    'batch_size' : hp.choice('batch_size',[20,50,80,100]),
    'activation0' : hp.choice('activation0',['tanh','relu']),
    'activation2' : hp.choice('activation1',['tanh','relu']),
    'activation1' : hp.choice('activation2',['tanh','relu'])
}

In [ ]:
# @markdown Performing Hyper-parameter tuning
trials=Trials()
tpe_best=fmin(createModel,
              space=space,
              algo=tpe.suggest,
              max_evals = 25,
              trials=trials,
              verbose=2)
loss=trials.best_trial['result']['loss']
best_params=space_eval(space,tpe_best)
print('Best Loss: {}'.format(loss))
print('Best Parameters by index: {}'.format(tpe_best))
print('Best Parameters: {}'.format(best_params))


Streaming output truncated to the last 5000 lines.

Epoch 23/100
20/72 [=======>......................]
 - ETA: 0s - loss: 1.5565e-05

72/72 [==============================]
 - 0s 581us/step - loss: 7.0793e-05

Epoch 24/100
20/72 [=======>......................]
 - ETA: 0s - loss: 1.2383e-05

72/72 [==============================]
 - 0s 645us/step - loss: 3.0807e-05

Epoch 25/100
20/72 [=======>......................]
 - ETA: 0s - loss: 1.7874e-05

72/72 [==============================]
 - 0s 772us/step - loss: 4.6758e-05

Epoch 26/100
20/72 [=======>......................]
 - ETA: 0s - loss: 7.5052e-06

72/72 [==============================]
 - 0s 630us/step - loss: 6.3936e-05

Epoch 27/100
20/72 [=======>................

In [ ]:
# @markdown Future 7 days Predictions
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#model=load_model("/content/models/modelwithloss557.h5")
lastdate = df_train['Date'][len(df_train.index)-1]
lastdate=lastdate.split('/')[0]
date=int(lastdate) + 1
df_test = df_train.tail(look_back)

for i in range(7):
  val = df_test['Confirmed Cases'].values
  val_index = df_test['Date'].values
  val = val.astype('float32')
  val = np.reshape(val, (-1, 1))
  scaler = MinMaxScaler(feature_range=(0, 1))
  val = scaler.fit_transform(val)
  X_future_test= []
  X_future_test.append(val[len(val)-look_back:])
  X_future_test =np.array(X_future_test)
  X_future_test = np.reshape(X_future_test, (X_future_test.shape[0], 1, X_future_test.shape[1]))
  future_predict = model.predict(X_future_test)
  future_predict = scaler.inverse_transform(future_predict)
  future_pred = list(future_predict.flatten())
  next_date = str(date) + '/05/20'
  date = date+1
  future_pred[0] = int(round(future_pred[0]))
  dict_train = {'Date': [next_date] , 'Confirmed Cases' :future_pred}
  future_df = pd.DataFrame(dict_train)
  df_test = df_test.append(future_df,ignore_index = True)

In [ ]:
future_pred

[1209190]

In [ ]:
df_test

,Date,Confirmed Cases
0,14/07/20,906752
1,15/07/20,936181
2,16/07/20,968876
3,17/07/20,1003832
4,18/07/20,1038716
5,19/07/20,1077618
6,20/07/20,1118043
7,21/07/20,1155191
8,22/07/20,1192915
9,23/07/20,1238635
